Author: Connor Marvin


# Pipeline V2


In [ ]:
#@title Install prerequisite this will take a while
# Install prerequisite, no need to run this if you are running the notebook from
# your laptop and you already install the packages in Anaconda

#!pip install biopython
%pip install "git+https://github.com/biopython/biopython.git@revert-4797-master"

!pip install ipytree
!pip install scikit-allel
!pip install zarr


#@title Mount Google Drive (You don't need to run this if you are running notebooks on your laptop)

from google.colab import drive

# The following command will prompt a URL for you to click and obtain the
# authorization code

drive.mount("/content/drive")


  Cloning https://github.com/biopython/biopython.git (to revision revert-4797-master) to /tmp/pip-req-build-vwh3f6mh
  Running command git clone --filter=blob:none --quiet https://github.com/biopython/biopython.git /tmp/pip-req-build-vwh3f6mh
  Running command git checkout -b revert-4797-master --track origin/revert-4797-master
  Switched to a new branch 'revert-4797-master'
  Branch 'revert-4797-master' set up to track remote branch 'revert-4797-master' from 'origin'.
  Resolved https://github.com/biopython/biopython.git to commit 337709ea99c32b9046430f90bb6efd196c7aa760
  Preparing metadata (setup.py) ... done
  Created wheel for biopython: filename=biopython-1.85.dev0-cp312-cp312-linux_x86_64.whl size=3122980 sha256=1364bd922dabeb88713a5e92bdc4e5561e080d54b53976b91233932e17f4a7d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-qfgd3rjs/wheels/70/b6/e3/5fb3a673d155b94973009cee8377d80b746e64584cd18f89ae
Successfully built biopython
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6

In [ ]:
# ============================================================
# PART 0: Imports + UID helper functions + GLOBAL SEQUENCE LIST
# ============================================================

from Bio import Entrez, SeqIO, AlignIO
from Bio.Align.Applications import MafftCommandline
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Align import PairwiseAligner
from Bio import Phylo

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

import numpy as np
import pandas as pd
import itertools

from pathlib import Path     # <<<<<<<<< THIS WAS MISSING

try:
    from IPython.display import display
except ImportError:
    def display(x):
        print(x)

# Path for the uniform master FASTA used by MAFFT
MASTER_FASTA_PATH = Path("sequences_master.fasta")

# ---------- UID helper functions ----------

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    d = 3
    while d * d <= n:
        if n % d == 0:
            return False
        d += 2
    return True

def choose_prime_for_sequence(length: int) -> int:
    for p in range(length // 20, 1, -1):
        if is_prime(p):
            return p
    for p in range(length, 1, -1):
        if is_prime(p):
            return p
    return 2

def compute_uid_from_sequence(seq_str: str) -> str:
    L = len(seq_str)
    if L == 0:
        return "00000.00000.00000"

    p = choose_prime_for_sequence(L)

    indices = []
    for i in range(1, 21):
        idx = i * p
        if idx <= L:
            indices.append(idx - 1)
        else:
            break

    sum_val = sum(ord(seq_str[i]) for i in indices)
    mod_val = L % p

    return f"{sum_val % 100000:05d}.{mod_val % 100000:05d}.00000"

# ---------- GLOBAL sequence containers ----------

all_records = []  # list of SeqRecord
all_ids = []      # list of ids
rec_id_to_uid = {}
uid_index = {}

# ---------- Master FASTA writer ----------

def write_master_fasta():
    safe_records = []
    for rec in all_records:
        safe_records.append(
            SeqRecord(
                rec.seq,
                id=rec.id,
                description=""
            )
        )
    SeqIO.write(safe_records, MASTER_FASTA_PATH, "fasta")
    print(f"MASTER FASTA: wrote {len(safe_records)} sequences to {MASTER_FASTA_PATH}")

# ---------- Add to global list and regenerate master FASTA ----------

def add_records_to_global(new_records):
    added = False
    for rec in new_records:
        if rec.id not in all_ids:
            all_records.append(rec)
            all_ids.append(rec.id)
            print(f"GLOBAL: added sequence {rec.id} (len={len(rec.seq)})")
            added = True
        else:
            print(f"GLOBAL: sequence {rec.id} already exists, skipping.")

    if added:
        write_master_fasta()
    else:
        print("GLOBAL: no new sequences added; master FASTA unchanged.")


/usr/local/lib/python3.12/dist-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [ ]:
print(all_records)

[]


In [ ]:
# ============================================================
# PART 1.1: Initialize / get sequence data from NCBI
#           and add them into the GLOBAL SEQUENCE LIST
#
# After this:
#   all_records / all_ids contain everything so far.
# ============================================================

Entrez.email = "cm4662@columbia.edu"  # required by NCBI

seq_ids = [
    'QRN78347.1',
    'QRX39425.1',
    'QUD52764.1',
    'QWE88920.1',
    'UFO69279.1',
    'UOZ45804.1',
    'UTM82166.1',
    'YP_009724390.1',
]

handle = Entrez.efetch(
    db="protein",
    id=",".join(seq_ids),
    rettype="fasta",
    retmode="text"
)
ncbi_records = list(SeqIO.parse(handle, "fasta"))
handle.close()

print("PART 1.1: Fetched from NCBI:")
for r in ncbi_records:
    print(" ", r.id, "length:", len(r.seq))

# Add to global sequence list
add_records_to_global(ncbi_records)


PART 1.1: Fetched from NCBI:
  QRN78347.1 length: 1270
  QRX39425.1 length: 1273
  QUD52764.1 length: 1271
  QWE88920.1 length: 1270
  UFO69279.1 length: 1270
  UOZ45804.1 length: 1268
  UTM82166.1 length: 1270
  YP_009724390.1 length: 1273
GLOBAL: added sequence QRN78347.1 (len=1270)
GLOBAL: added sequence QRX39425.1 (len=1273)
GLOBAL: added sequence QUD52764.1 (len=1271)
GLOBAL: added sequence QWE88920.1 (len=1270)
GLOBAL: added sequence UFO69279.1 (len=1270)
GLOBAL: added sequence UOZ45804.1 (len=1268)
GLOBAL: added sequence UTM82166.1 (len=1270)
GLOBAL: added sequence YP_009724390.1 (len=1273)
MASTER FASTA: wrote 8 sequences to sequences_master.fasta


In [ ]:
# ============================================================
# PART 1.2: Convert metadata+sequence .txt file into a FASTA
#           and auto-generate UID if it is "NOTSETYET".
#           Then add the sequence to GLOBAL SEQUENCE LIST.
#
# Returns:
#   fasta_path, meta_dict, sequence_string
#   (and updates all_records / all_ids)
# ============================================================

def txt_with_meta_to_fasta(txt_path, fasta_path=None):
    txt_path = Path(txt_path)
    if fasta_path is None:
        fasta_path = txt_path.with_suffix(".fasta")

    meta = {}
    seq_lines = []
    in_seq = False

    with open(txt_path, "r") as f:
        for line in f:
            line = line.rstrip("\n")

            if not in_seq and line.strip() == "":
                continue

            # Detect start of sequence block
            if not in_seq and line.strip().lower().startswith("sequence:"):
                in_seq = True
                continue

            if not in_seq:
                if ":" in line:
                    key, val = line.split(":", 1)
                    meta[key.strip()] = val.strip()
            else:
                seq_lines.append(line.strip().replace(" ", ""))

    seq = "".join(seq_lines)
    if not seq:
        raise ValueError(f"No sequence found in {txt_path}")

    # UID handling: generate if NOTSETYET
    uid_val = meta.get("uid", "NOTSETYET")
    if uid_val == "NOTSETYET":
        new_uid = compute_uid_from_sequence(seq)
        meta["uid"] = new_uid
        print(f"PART 1.2: UID was NOTSETYET, computed UID = {new_uid}")
    else:
        print(f"PART 1.2: Using existing UID = {uid_val}")

    # Primary FASTA ID: ncbi_id if present, otherwise use uid
    primary_id = meta.get("ncbi_id", meta["uid"])

    # Build FASTA header:
    # >primary_id uid=...|variant=...|...|ncbi_id=...
    meta_parts = [f"{k}={v}" for k, v in meta.items()]
    meta_str = "|".join(meta_parts)
    fasta_header = f">{primary_id} {meta_str}"

    with open(fasta_path, "w") as out:
        out.write(fasta_header + "\n")
        for i in range(0, len(seq), 60):
            out.write(seq[i:i+60] + "\n")

    print(f"PART 1.2: Wrote FASTA to {fasta_path}")

    # Parse back into SeqRecord(s) and add to global list
    new_records = list(SeqIO.parse(fasta_path, "fasta"))
    add_records_to_global(new_records)

    return fasta_path, meta, seq

# Example usage for a doctor-provided file:
#

In [ ]:
# ============================================================
# PART 1.3: Import raw FASTA file(s) and add to global list
#
# Uses:
#   - add_records_to_global (from PART 0)
#   - all_records / all_ids (global containers)
#
# Behavior:
#   - Reads the given FASTA file
#   - Optionally prefixes IDs to avoid collisions
#   - Adds all sequences to the global list
#   - Automatically regenerates sequences_master.fasta
# ============================================================

from pathlib import Path
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

def import_raw_fasta(fasta_path, id_prefix=None):
    """
    Import sequences from a raw FASTA file and add them to the global list.

    Parameters
    ----------
    fasta_path : str or Path
        Path to the FASTA file you just uploaded.
    id_prefix : str or None
        Optional string to prepend to each sequence ID, e.g. "sample1_".
        Useful if you're worried about ID collisions.

    Effects
    -------
    - Populates/extends all_records and all_ids (via add_records_to_global)
    - Triggers write_master_fasta() through add_records_to_global
    """
    fasta_path = Path(fasta_path)

    if not fasta_path.exists():
        raise FileNotFoundError(f"FASTA file not found: {fasta_path}")

    raw_records = list(SeqIO.parse(fasta_path, "fasta"))
    if not raw_records:
        print(f"PART 1.3: No sequences found in {fasta_path}")
        return

    print(f"PART 1.3: Found {len(raw_records)} sequence(s) in {fasta_path.name}:")

    new_records = []
    for rec in raw_records:
        old_id = rec.id

        # Optionally prefix ID
        if id_prefix is not None:
            new_id = f"{id_prefix}{old_id}"
        else:
            new_id = old_id

        # Create a clean SeqRecord with new_id and empty description
        clean_rec = SeqRecord(
            rec.seq,
            id=new_id,
            description=""
        )
        new_records.append(clean_rec)
        print(f"  - {old_id} -> {clean_rec.id} (len={len(clean_rec.seq)})")

    # Add to global list and regenerate master FASTA
    add_records_to_global(new_records)

    print("PART 1.3: Import complete.\n")

# Example usage:
# import_raw_fasta("new_sample.fasta")
# import_raw_fasta("new_sample.fasta", id_prefix="patient001_")


In [ ]:
fasta_path, meta, seq = txt_with_meta_to_fasta("eggsalad1.txt")


PART 1.2: UID was NOTSETYET, computed UID = 01523.00049.00000
PART 1.2: Wrote FASTA to eggsalad1.fasta
GLOBAL: added sequence eggsalad1.egg (len=1269)
MASTER FASTA: wrote 9 sequences to sequences_master.fasta


In [ ]:
import_raw_fasta("sequence (16) - mutant 3.fasta")

PART 1.3: Found 1 sequence(s) in sequence (16) - mutant 3.fasta:
  -  ->  (len=1270)
GLOBAL: added sequence  (len=1270)
MASTER FASTA: wrote 12 sequences to sequences_master.fasta
PART 1.3: Import complete.



In [ ]:
fasta_path, meta, seq = txt_with_meta_to_fasta("eggsalad2.txt")

PART 1.2: UID was NOTSETYET, computed UID = 01523.00049.00000
PART 1.2: Wrote FASTA to eggsalad2.fasta
GLOBAL: added sequence eggsalad2.egg (len=1269)
MASTER FASTA: wrote 10 sequences to sequences_master.fasta


In [ ]:
# ============================================================
# PART 2: Store seq data in files with extra info + UID
#
# Uses:
#   all_records / all_ids  (global list)
#
# For EACH sequence in all_records:
#   - <uid>_<id>.fasta   (metadata in header, UID first)
#   - <uid>_<id>.txt     (metadata + sequence)
#
# Populates:
#   rec_id_to_uid : dict mapping primary ID -> UID
#   uid_index     : dict[(sum_part, mod_part)] -> uid
# ============================================================

# Placeholder metadata defaults (replace with real inputs later)
VARIANT_NAME    = "__________"
SEQUENCE_AGE    = "02122025"            # ddmmyyyy
UPLOAD_DATE     = "02122025"            # ddmmyyyy
PRACT_UID       = "TESTPR"
PATIENT_UID     = "TESTPA"
EMAIL           = "doctor@example.com"
PHONE           = "845-321-1123"
FLAG            = "OK"

# Reset mappings (rebuild from all_records each time Part 2 runs)
rec_id_to_uid.clear()
uid_index.clear()

out_dir = Path("sequence_files")
out_dir.mkdir(exist_ok=True)

for rec in all_records:
    seq_str = str(rec.seq)

    # Compute UID from sequence
    uid = compute_uid_from_sequence(seq_str)
    rec_id_to_uid[rec.id] = uid

    sum_part_str, mod_part_str, third_part_str = uid.split(".")
    sum_part = int(sum_part_str)
    mod_part = int(mod_part_str)
    uid_index[(sum_part, mod_part)] = uid

    # IMPORTANT: ensure UID is first field in metadata
    meta = {
        "uid": uid,
        "variant": VARIANT_NAME,
        "sequence_age": SEQUENCE_AGE,
        "upload_date": UPLOAD_DATE,
        "practitioner_uid": PRACT_UID,
        "patient_uid": PATIENT_UID,
        "email": EMAIL,
        "phone": PHONE,
        "flag": FLAG,
        "ncbi_id": rec.id,   # keep NCBI (or primary) ID in metadata
    }

    # FASTA header is now ONLY the key=value metadata string,
    # starting with uid=..., NOT ">NCBIID ...".
    header_str = "|".join(f"{k}={v}" for k, v in meta.items())
    fasta_header = f">{header_str}"

    fasta_path = out_dir / f"{uid}_{rec.id}.fasta"
    with open(fasta_path, "w") as f:
        f.write(fasta_header + "\n")
        for i in range(0, len(seq_str), 60):
            f.write(seq_str[i:i+60] + "\n")

    # Plain text file
    txt_path = out_dir / f"{uid}_{rec.id}.txt"
    with open(txt_path, "w") as f:
        for k, v in meta.items():
            f.write(f"{k}: {v}\n")
        f.write("sequence:\n")
        f.write(seq_str + "\n")

    print(f"PART 2: Written {fasta_path.name} and {txt_path.name}  (UID={uid})")


PART 2: Written 01573.00050.00000_QRN78347.1.fasta and 01573.00050.00000_QRN78347.1.txt  (UID=01573.00050.00000)
PART 2: Written 01565.00053.00000_QRX39425.1.fasta and 01565.00053.00000_QRX39425.1.txt  (UID=01565.00053.00000)
PART 2: Written 01592.00051.00000_QUD52764.1.fasta and 01592.00051.00000_QUD52764.1.txt  (UID=01592.00051.00000)
PART 2: Written 01568.00050.00000_QWE88920.1.fasta and 01568.00050.00000_QWE88920.1.txt  (UID=01568.00050.00000)
PART 2: Written 01597.00050.00000_UFO69279.1.fasta and 01597.00050.00000_UFO69279.1.txt  (UID=01597.00050.00000)
PART 2: Written 01538.00048.00000_UOZ45804.1.fasta and 01538.00048.00000_UOZ45804.1.txt  (UID=01538.00048.00000)
PART 2: Written 01592.00050.00000_UTM82166.1.fasta and 01592.00050.00000_UTM82166.1.txt  (UID=01592.00050.00000)
PART 2: Written 01565.00053.00000_YP_009724390.1.fasta and 01565.00053.00000_YP_009724390.1.txt  (UID=01565.00053.00000)
PART 2: Written 01523.00049.00000_eggsalad1.egg.fasta and 01523.00049.00000_eggsalad1.eg

In [ ]:
print(all_records)

[SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='QRN78347.1', name='QRN78347.1', description='QRN78347.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]', dbxrefs=[]), SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNFTNRTQLPSAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='QRX39425.1', name='QRX39425.1', description='QRX39425.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]', dbxrefs=[]), SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='QUD52764.1', name='QUD52764.1', description='QUD52764.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]', dbxrefs=[]), SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='QWE88920.1', name='QWE88920.1', description='QWE88920.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]', dbxrefs=[]), SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), 

In [ ]:
# ============================================================
# PART 3: Global pairwise alignment scores ONLY (no MAFFT)
#         Scoring: match = +1, mismatch = -1, gap = -2
#
# Uses:
#   all_records, all_ids   (from global list)
#   rec_id_to_uid          (from Part 2)
#
# Produces:
#   global_score_df        (raw scores, indexed by IDs)
#   norm_score_df          (normalized scores, indexed by IDs)
#   global_score_uid_df    (raw scores, indexed by UIDs)
#   norm_score_uid_df      (normalized scores, indexed by UIDs)
# ============================================================

from Bio.Align import PairwiseAligner
import numpy as np
import pandas as pd

if len(all_records) == 0:
    raise RuntimeError("PART 3: No sequences in global list. Run Part 1.1 and/or 1.2 first.")

print("PART 3: Pairwise global scoring across", len(all_records), "sequences:")
for r in all_records:
    print(" ", r.id, "length:", len(r.seq))

# Configure pairwise global aligner
aligner = PairwiseAligner()
aligner.mode = "global"
aligner.match_score = 1.0
aligner.mismatch_score = -1.0
aligner.open_gap_score = -2.0
aligner.extend_gap_score = -2.0

n = len(all_records)

# Raw score matrix (IDs x IDs)
global_score_df = pd.DataFrame(
    np.zeros((n, n), dtype=float),
    index=all_ids,
    columns=all_ids,
)

# Normalized score matrix (score per residue)
norm_score_df = pd.DataFrame(
    np.zeros((n, n), dtype=float),
    index=all_ids,
    columns=all_ids,
)

# Fill matrices (symmetrically)
for i, rec_i in enumerate(all_records):
    seq_i = rec_i.seq
    for j, rec_j in enumerate(all_records):
        if j < i:
            # mirror
            global_score_df.iat[i, j] = global_score_df.iat[j, i]
            norm_score_df.iat[i, j] = norm_score_df.iat[j, i]
        else:
            raw_score = aligner.score(seq_i, rec_j.seq)
            global_score_df.iat[i, j] = raw_score

            max_len = max(len(seq_i), len(rec_j.seq))
            norm_score_df.iat[i, j] = raw_score / max_len if max_len > 0 else 0.0

print("\nRaw global alignment score matrix (IDs, match=+1, mismatch=-1, gap=-2):")
display(global_score_df)

print("\nNormalized global alignment score matrix (IDs, score per residue):")
display(norm_score_df)

# UID-indexed copies (for later / PART 4)
uid_list = [rec_id_to_uid.get(rid, "NOUID") for rid in all_ids]

global_score_uid_df = global_score_df.copy()
global_score_uid_df.index = uid_list
global_score_uid_df.columns = uid_list

norm_score_uid_df = norm_score_df.copy()
norm_score_uid_df.index = uid_list
norm_score_uid_df.columns = uid_list

print("\nRaw global alignment score matrix (UIDs):")
display(global_score_uid_df)

print("\nNormalized global alignment score matrix (UIDs):")
display(norm_score_uid_df)

# Most similar sequence per ID (for inspection)
print("\nMost similar sequence for each (based on RAW scores, IDs):\n")
for seq_id in global_score_df.index:
    row = global_score_df.loc[seq_id].drop(labels=[seq_id])
    if row.empty:
        print(f"{seq_id} -> no other sequences to compare.")
    else:
        best_match_id = row.idxmax()
        best_score = row.max()
        print(f"{seq_id} -> {best_match_id}  (score = {best_score:.2f})")

print("\nMost similar sequence for each (based on NORMALIZED scores, IDs):\n")
for seq_id in norm_score_df.index:
    row = norm_score_df.loc[seq_id].drop(labels=[seq_id])
    if row.empty:
        print(f"{seq_id} -> no other sequences to compare.")
    else:
        best_match_id = row.idxmax()
        best_score = row.max()
        print(f"{seq_id} -> {best_match_id}  (normalized score = {best_score:.4f})")


PART 3: Pairwise global scoring across 10 sequences:
  QRN78347.1 length: 1270
  QRX39425.1 length: 1273
  QUD52764.1 length: 1271
  QWE88920.1 length: 1270
  UFO69279.1 length: 1270
  UOZ45804.1 length: 1268
  UTM82166.1 length: 1270
  YP_009724390.1 length: 1273
  eggsalad1.egg length: 1269
  eggsalad2.egg length: 1269

Raw global alignment score matrix (IDs, match=+1, mismatch=-1, gap=-2):


,QRN78347.1,QRX39425.1,QUD52764.1,QWE88920.1,UFO69279.1,UOZ45804.1,UTM82166.1,YP_009724390.1,eggsalad1.egg,eggsalad2.egg
QRN78347.1,1270.0,1240.0,1232.0,1235.0,1180.0,1189.0,1187.0,1250.0,1265.0,1213.0
QRX39425.1,1240.0,1273.0,1231.0,1234.0,1181.0,1194.0,1192.0,1249.0,1235.0,1189.0
QUD52764.1,1232.0,1231.0,1271.0,1234.0,1177.0,1192.0,1188.0,1251.0,1227.0,1177.0
QWE88920.1,1235.0,1234.0,1234.0,1270.0,1193.0,1197.0,1185.0,1250.0,1230.0,1178.0
UFO69279.1,1180.0,1181.0,1177.0,1193.0,1270.0,1207.0,1199.0,1189.0,1175.0,1123.0
UOZ45804.1,1189.0,1194.0,1192.0,1197.0,1207.0,1268.0,1242.0,1198.0,1184.0,1144.0
UTM82166.1,1187.0,1192.0,1188.0,1185.0,1199.0,1242.0,1270.0,1196.0,1182.0,1142.0
YP_009724390.1,1250.0,1249.0,1251.0,1250.0,1189.0,1198.0,1196.0,1273.0,1245.0,1193.0
eggsalad1.egg,1265.0,1235.0,1227.0,1230.0,1175.0,1184.0,1182.0,1245.0,1269.0,1215.0
eggsalad2.egg,1213.0,1189.0,1177.0,1178.0,1123.0,1144.0,1142.0,1193.0,1215.0,1269.0



Normalized global alignment score matrix (IDs, score per residue):


,QRN78347.1,QRX39425.1,QUD52764.1,QWE88920.1,UFO69279.1,UOZ45804.1,UTM82166.1,YP_009724390.1,eggsalad1.egg,eggsalad2.egg
QRN78347.1,1.000000,0.974077,0.969315,0.972441,0.929134,0.936220,0.934646,0.981932,0.996063,0.955118
QRX39425.1,0.974077,1.000000,0.967007,0.969364,0.927730,0.937942,0.936371,0.981147,0.970149,0.934014
QUD52764.1,0.969315,0.967007,1.000000,0.970889,0.926042,0.937844,0.934697,0.982718,0.965382,0.926042
QWE88920.1,0.972441,0.969364,0.970889,1.000000,0.939370,0.942520,0.933071,0.981932,0.968504,0.927559
UFO69279.1,0.929134,0.927730,0.926042,0.939370,1.000000,0.950394,0.944094,0.934014,0.925197,0.884252
UOZ45804.1,0.936220,0.937942,0.937844,0.942520,0.950394,1.000000,0.977953,0.941084,0.933018,0.901497
UTM82166.1,0.934646,0.936371,0.934697,0.933071,0.944094,0.977953,1.000000,0.939513,0.930709,0.899213
YP_009724390.1,0.981932,0.981147,0.982718,0.981932,0.934014,0.941084,0.939513,1.000000,0.978005,0.937156
eggsalad1.egg,0.996063,0.970149,0.965382,0.968504,0.925197,0.933018,0.930709,0.978005,1.000000,0.957447
eggsalad2.egg,0.955118,0.934014,0.926042,0.927559,0.884252,0.901497,0.899213,0.937156,0.957447,1.000000



Raw global alignment score matrix (UIDs):


,01573.00050.00000,01565.00053.00000,01592.00051.00000,01568.00050.00000,01597.00050.00000,01538.00048.00000,01592.00050.00000,01565.00053.00000,01523.00049.00000,01523.00049.00000
01573.00050.00000,1270.0,1240.0,1232.0,1235.0,1180.0,1189.0,1187.0,1250.0,1265.0,1213.0
01565.00053.00000,1240.0,1273.0,1231.0,1234.0,1181.0,1194.0,1192.0,1249.0,1235.0,1189.0
01592.00051.00000,1232.0,1231.0,1271.0,1234.0,1177.0,1192.0,1188.0,1251.0,1227.0,1177.0
01568.00050.00000,1235.0,1234.0,1234.0,1270.0,1193.0,1197.0,1185.0,1250.0,1230.0,1178.0
01597.00050.00000,1180.0,1181.0,1177.0,1193.0,1270.0,1207.0,1199.0,1189.0,1175.0,1123.0
01538.00048.00000,1189.0,1194.0,1192.0,1197.0,1207.0,1268.0,1242.0,1198.0,1184.0,1144.0
01592.00050.00000,1187.0,1192.0,1188.0,1185.0,1199.0,1242.0,1270.0,1196.0,1182.0,1142.0
01565.00053.00000,1250.0,1249.0,1251.0,1250.0,1189.0,1198.0,1196.0,1273.0,1245.0,1193.0
01523.00049.00000,1265.0,1235.0,1227.0,1230.0,1175.0,1184.0,1182.0,1245.0,1269.0,1215.0
01523.00049.00000,1213.0,1189.0,1177.0,1178.0,1123.0,1144.0,1142.0,1193.0,1215.0,1269.0



Normalized global alignment score matrix (UIDs):


,01573.00050.00000,01565.00053.00000,01592.00051.00000,01568.00050.00000,01597.00050.00000,01538.00048.00000,01592.00050.00000,01565.00053.00000,01523.00049.00000,01523.00049.00000
01573.00050.00000,1.000000,0.974077,0.969315,0.972441,0.929134,0.936220,0.934646,0.981932,0.996063,0.955118
01565.00053.00000,0.974077,1.000000,0.967007,0.969364,0.927730,0.937942,0.936371,0.981147,0.970149,0.934014
01592.00051.00000,0.969315,0.967007,1.000000,0.970889,0.926042,0.937844,0.934697,0.982718,0.965382,0.926042
01568.00050.00000,0.972441,0.969364,0.970889,1.000000,0.939370,0.942520,0.933071,0.981932,0.968504,0.927559
01597.00050.00000,0.929134,0.927730,0.926042,0.939370,1.000000,0.950394,0.944094,0.934014,0.925197,0.884252
01538.00048.00000,0.936220,0.937942,0.937844,0.942520,0.950394,1.000000,0.977953,0.941084,0.933018,0.901497
01592.00050.00000,0.934646,0.936371,0.934697,0.933071,0.944094,0.977953,1.000000,0.939513,0.930709,0.899213
01565.00053.00000,0.981932,0.981147,0.982718,0.981932,0.934014,0.941084,0.939513,1.000000,0.978005,0.937156
01523.00049.00000,0.996063,0.970149,0.965382,0.968504,0.925197,0.933018,0.930709,0.978005,1.000000,0.957447
01523.00049.00000,0.955118,0.934014,0.926042,0.927559,0.884252,0.901497,0.899213,0.937156,0.957447,1.000000



Most similar sequence for each (based on RAW scores, IDs):

QRN78347.1 -> eggsalad1.egg  (score = 1265.00)
QRX39425.1 -> YP_009724390.1  (score = 1249.00)
QUD52764.1 -> YP_009724390.1  (score = 1251.00)
QWE88920.1 -> YP_009724390.1  (score = 1250.00)
UFO69279.1 -> UOZ45804.1  (score = 1207.00)
UOZ45804.1 -> UTM82166.1  (score = 1242.00)
UTM82166.1 -> UOZ45804.1  (score = 1242.00)
YP_009724390.1 -> QUD52764.1  (score = 1251.00)
eggsalad1.egg -> QRN78347.1  (score = 1265.00)
eggsalad2.egg -> eggsalad1.egg  (score = 1215.00)

Most similar sequence for each (based on NORMALIZED scores, IDs):

QRN78347.1 -> eggsalad1.egg  (normalized score = 0.9961)
QRX39425.1 -> YP_009724390.1  (normalized score = 0.9811)
QUD52764.1 -> YP_009724390.1  (normalized score = 0.9827)
QWE88920.1 -> YP_009724390.1  (normalized score = 0.9819)
UFO69279.1 -> UOZ45804.1  (normalized score = 0.9504)
UOZ45804.1 -> UTM82166.1  (normalized score = 0.9780)
UTM82166.1 -> UOZ45804.1  (normalized score = 0.9780)
YP_0097243

In [ ]:
# ============================================================
# PART 4: Annotate per-sequence UID FASTAs with:
#   - variant_match_uid : UID of most similar sequence
#   - align_score_raw   : raw global alignment score to that match
#   - align_score_norm  : normalized score to that match
#
# Uses:
#   all_ids              (global IDs)
#   rec_id_to_uid        (from Part 2)
#   global_score_df      (from Part 3)
#   norm_score_df        (from Part 3)
#   out_dir              (sequence_files directory from Part 2)
#
# Assumes per-sequence FASTA headers are of the form:
#   >uid=...|variant=...|...|ncbi_id=...
# ============================================================

if len(all_ids) < 2:
    print("PART 4: Only one sequence in the global list; nothing to annotate with similarity.")
else:
    for seq_id in all_ids:
        # Row of scores against all others
        row_raw = global_score_df.loc[seq_id]

        # Drop self-comparison
        row_raw_no_self = row_raw.drop(labels=[seq_id])
        if row_raw_no_self.empty:
            print(f"PART 4: {seq_id} has no other sequences to compare; skipping.")
            continue

        # Best matching sequence (highest raw score)
        best_match_id = row_raw_no_self.idxmax()
        raw_score = row_raw_no_self.max()
        norm_score = norm_score_df.loc[seq_id, best_match_id]

        # UIDs for this sequence and its best match
        this_uid = rec_id_to_uid.get(seq_id, None)
        match_uid = rec_id_to_uid.get(best_match_id, None)

        if this_uid is None:
            print(f"PART 4: No UID found for {seq_id}; skipping FASTA update.")
            continue
        if match_uid is None:
            print(f"PART 4: No UID found for best match {best_match_id}; skipping FASTA update for {seq_id}.")
            continue

        # Path to this sequence's FASTA file (from Part 2 naming convention)
        fasta_path = out_dir / f"{this_uid}_{seq_id}.fasta"
        if not fasta_path.exists():
            print(f"PART 4: FASTA file not found for {seq_id} at {fasta_path}; skipping.")
            continue

        # Read existing FASTA content
        with open(fasta_path, "r") as f:
            lines = f.readlines()
        if not lines:
            print(f"PART 4: Empty FASTA file {fasta_path}; skipping.")
            continue

        header = lines[0].rstrip("\n")
        if not header.startswith(">"):
            print(f"PART 4: Invalid FASTA header in {fasta_path}; skipping.")
            continue

        # Strip leading '>'
        header_body = header[1:].strip()

        # Entire header_body is a meta string:
        #   uid=...|variant=...|...|ncbi_id=...
        meta_dict = {}
        if header_body:
            for chunk in header_body.split("|"):
                if "=" in chunk:
                    k, v = chunk.split("=", 1)
                    meta_dict[k.strip()] = v.strip()

        # Add / overwrite our similarity fields
        meta_dict["variant_match_uid"] = match_uid
        meta_dict["align_score_raw"] = f"{raw_score:.4f}"
        meta_dict["align_score_norm"] = f"{norm_score:.4f}"

        # Rebuild meta string (UID stays first because it was inserted first in Part 2)
        meta_str = "|".join(f"{k}={v}" for k, v in meta_dict.items())

        # New header line
        new_header = f">{meta_str}"

        # Write back updated FASTA
        lines[0] = new_header + "\n"
        with open(fasta_path, "w") as f:
            f.writelines(lines)

        print(
            f"PART 4: Updated {fasta_path.name} with "
            f"variant_match_uid={match_uid}, "
            f"align_score_raw={raw_score:.4f}, "
            f"align_score_norm={norm_score:.4f}"
        )


PART 4: Updated 01573.00050.00000_QRN78347.1.fasta with variant_match_uid=01523.00049.00000, align_score_raw=1265.0000, align_score_norm=0.9961
PART 4: Updated 01565.00053.00000_QRX39425.1.fasta with variant_match_uid=01565.00053.00000, align_score_raw=1249.0000, align_score_norm=0.9811
PART 4: Updated 01592.00051.00000_QUD52764.1.fasta with variant_match_uid=01565.00053.00000, align_score_raw=1251.0000, align_score_norm=0.9827
PART 4: Updated 01568.00050.00000_QWE88920.1.fasta with variant_match_uid=01565.00053.00000, align_score_raw=1250.0000, align_score_norm=0.9819
PART 4: Updated 01597.00050.00000_UFO69279.1.fasta with variant_match_uid=01538.00048.00000, align_score_raw=1207.0000, align_score_norm=0.9504
PART 4: Updated 01538.00048.00000_UOZ45804.1.fasta with variant_match_uid=01592.00050.00000, align_score_raw=1242.0000, align_score_norm=0.9780
PART 4: Updated 01592.00050.00000_UTM82166.1.fasta with variant_match_uid=01538.00048.00000, align_score_raw=1242.0000, align_score_norm

EGG2

Based on the alignment, using the reference sequence `YP_009724390.1` as the reference sequence, we can identify all mutations in spike proteins in the other 7 sequences. This can be done similarly to what we did for influenza hemagglutinin sequences in class.

Once you identify the mutations for each sequence in the spike protein, you can assign their variant identity (e.g. alpha, delta, omicron) based on the signature given in the following plot from ![Viralzon](https://viralzone.expasy.org/9556):

![](https://viralzone.expasy.org/resources/Variants_graph.svg)